In [1]:
import requests
import json
import pandas as pd

In [2]:
# make selection of months (last month in list won't be requested)

months = [] + \
["2016-" + str(month) for month in range(11,13)] + \
["2017-0" + str(month) for month in range(1,10)] + \
["2017-" + str(month) for month in range(10,13)] + \
["2018-0" + str(month) for month in range(1,10)] + \
["2018-" + str(month) for month in range(10,13)] + \
["2019-0" + str(month) for month in range(1,10)] + \
["2019-" + str(month) for month in range(10,11)]

# micro choice of months for testing
#months = ("2019-07", "2019-08", "2019-09")

In [3]:
# get topic names from Augmento
topics = requests.request("Get", url="http://api-dev.augmento.ai/v0.1/topics")
topics = json.loads(topics.content)
topics = list(topics.values())

In [4]:
# prepare request of data from Augmento
# bin_size = 24H / 1H

def request_and_clean(source, bin_size="24H", coin='bitcoin', months=months):
    
    url = "http://api-dev.augmento.ai/v0.1/events/aggregated"
    headers = {}
    idx_lastmonth = len(months) - 1
    
    # initialize dataframe
    data_requested = pd.DataFrame()
    
    # loop over months and request data
    for idx, month in enumerate(months):
        if idx==idx_lastmonth: break

        params = {
            "source" : source,
            "coin" : coin,
            "bin_size" : bin_size,
            "count_ptr" : 1000,
            "start_ptr" : 0,
            "start_datetime" : months[idx]+"-01T00:00:00Z",
            "end_datetime" : months[idx+1]+"-01T00:00:00Z",
        }

        r = requests.request("GET", url, params=params, headers=headers)
        j = json.loads(r.content)
        df = pd.DataFrame(j)
        data_requested = data_requested.append(df)
        
    # unnest topic counts from requested data, change col names
    counts = data_requested["counts"].apply(pd.Series)
    counts.columns = topics
    
    # clean data a bit
    data_wide = pd.concat((data_requested["datetime"], counts[:]), axis=1)
    data_wide.index = pd.to_datetime(data_wide["datetime"], format='%Y-%m-%dT%H:%M:%SZ')
    data_wide = data_wide.drop("datetime", axis=1)
    
    return(data_wide)

In [5]:
# request and clean data for bin_size = 24H (default)

data_twitter = request_and_clean('twitter')
data_reddit = request_and_clean('reddit')
data_bitcointalk = request_and_clean('bitcointalk')

# sum by position (dataframes are identical)
data24h = data_twitter.add(data_reddit)
data24h = data24h.add(data_bitcointalk)

In [6]:
# request and clean data for bin_size = 1H (is this required for further steps ???)

data_twitter = request_and_clean('twitter', bin_size='1H')
data_reddit = request_and_clean('reddit', bin_size='1H')
data_bitcointalk = request_and_clean('bitcointalk', bin_size='1H')

# sum by position (dataframes are identical)
data1h = data_twitter.add(data_reddit)
data1h = data1h.add(data_bitcointalk)

In [ ]:
# check if all good:

#data_wide

#data_wide.describe()
#data_wide.dtypes

In [7]:
# prepare volume and price data 24h
# source: https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20130429&end=20200129

btc = pd.read_csv("btc_data.csv", sep=";", thousands=".", decimal=",")

btc.Date = pd.to_datetime(btc["Date"])
btc = btc.rename(columns = {"Close**":"BTC_Price", "Volume":"BTC_Volume"})
btc = btc.drop(columns = ["Open*", "High", "Low", "Market Cap"])


print(len(btc.axes[0]), "rows in total", "\n",
sum(btc.BTC_Price == 0), "where price is missing", "\n",
sum(btc.BTC_Volume == 0), "where volume is missing", "\n",
"These are all at the end of the dataset, so we can just omit them: \n")
print(btc.Date[btc.BTC_Volume == 0])


# omit rows w/ missing volume
btc = btc[btc.BTC_Volume > 0]

# join datasets
data24h2 = pd.merge(left=btc, right=data24h, left_on="Date", right_index=True)
data24h2 = data24h2.reset_index(drop=True)

2466 rows in total 
 0 where price is missing 
 242 where volume is missing 
 These are all at the end of the dataset, so we can just omit them: 

2224   2013-12-26
2225   2013-12-25
2226   2013-12-24
2227   2013-12-23
2228   2013-12-22
          ...    
2461   2013-05-03
2462   2013-05-02
2463   2013-05-01
2464   2013-04-30
2465   2013-04-29
Name: Date, Length: 242, dtype: datetime64[ns]


In [40]:
# prepare price data 1h
# source: http://www.cryptodatadownload.com/cdd/Coinbase_BTCUSD_h.csv
# via http://www.cryptodatadownload.com/data/northamerican/ 

btc_price = pd.read_csv("Coinbase_BTCUSD_1h.csv", skiprows=1)
btc_price.index = pd.to_datetime(btc_price["Date"], format='%Y-%m-%d %I-%p')
btc_price = btc_price.rename(columns={'Close':'BTC_Price'})


# merge topics and prices
data1h2 = pd.merge(left=btc_price[["BTC_Price"]], right=data1h, left_index=True, right_index=True)
data1h2 = data1h2.reset_index().rename(columns={"index":"datetime"}) # sort_index()

print(len(data1h.axes[0]) - len(data1h2.axes[0]), "lines excluded as no price available")

5819 lines excluded as no price available


In [41]:
print("The created dataset for 24h base has", len(data24h2.axes[0]), "rows and", len(data24h2.axes[1]), "variables.")
print("The created dataset for 1h base has", len(data1h2.axes[0]), "rows and", len(data1h2.axes[1]), "variables.")

The created dataset for 24h base has 1064 rows and 96 variables.
The created dataset for 1h base has 19717 rows and 95 variables.


In [43]:
# save data

data1h2.to_csv('../augmento_BTC_1h.csv', index=False)
data24h2.to_csv('../augmento_BTC_24h.csv', index=False)
data_twitter.to_csv('../augmento_BTC_1h_twitter.csv', index=False)